# RTX 4070 GPU 환경에서 우울증 판별 모델 개발

이 노트북은 **NVIDIA GeForce RTX 4070**과 같이 VRAM이 약 8GB인 환경을 고려하여, 우울증 여부를 판별하는 이진 분류 모델을 학습하는 과정을 제공합니다. 앞서 설명한 데이터 구조와 감정 재분류를 그대로 사용하되, **작은 메모리 환경**에서도 학습이 가능하도록 배치 크기를 줄이고 혼합정밀도 학습(mixed precision training)을 적용합니다.

데이터는 다음과 같은 구조를 가정합니다:
```
data/
  train/
    train_image/
    train_label/
  vali/
    vali_image/
    vali_label/
```


## 1. 패키지 불러오기와 기본 설정

딥러닝 학습에 필요한 라이브러리들을 불러옵니다. RTX 4070 환경에서는 메모리가 한정적이므로 **혼합정밀도 학습(mixed precision training)**을 사용하여 메모리 사용량을 줄입니다. 이를 위해 `torch.cuda.amp` 모듈을 가져옵니다.


In [ ]:
import os
import json
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
import torchvision.models as models

# 혼합정밀도 학습을 위한 모듈
from torch.cuda.amp import autocast, GradScaler

# 현재 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


## 2. 감정 레이블 정의

감정 레이블을 한국어로 매핑하고 우울/비우울 두 클래스로 재분류합니다.


In [ ]:
EMOTIONS = {
    'anger': '분노',
    'anxiety': '불안',
    'hurt': '상처',
    'joy': '기쁨',
    'neutral': '중립',
    'sadness': '슬픔',
    'surprise': '당황'
}
DEPRESSION_EMOTIONS = ['anxiety', 'hurt', 'sadness']
NON_DEPRESSION_EMOTIONS = ['anger', 'joy', 'neutral', 'surprise']


## 3. 데이터셋 클래스와 전처리

앞선 노트북과 동일하게 JSON 레이블을 파싱하여 (이미지 경로, 라벨) 리스트를 만들고, `EmotionDataset` 클래스로 데이터를 로드합니다. 혼합정밀도 학습은 데이터셋 정의와는 무관하므로 여기서는 동일한 구현을 사용합니다.


In [ ]:
from PIL import Image

def parse_label_files(label_dir, image_dir):
    """
    JSON 라벨 파일을 읽어 이미지 경로와 이진 라벨(우울=1, 비우울=0)을 반환합니다.
    """
    samples = []
    json_files = sorted(glob(os.path.join(label_dir, '*.json')))
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for file_name, emotion in data.items():
            img_path = os.path.join(image_dir, emotion, file_name)
            label = 1 if emotion in DEPRESSION_EMOTIONS else 0
            samples.append((img_path, label))
    return samples

class EmotionDataset(Dataset):
    """
    이미지 경로와 라벨을 받아 PyTorch Tensor로 변환하여 반환하는 데이터셋 클래스.
    """
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)


## 4. 데이터 전처리(변환) 정의

학습 데이터에는 데이터 증강을 포함하고, 검증 데이터에는 기본 전처리만 적용합니다.


In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

vali_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


## 5. 데이터셋 로딩과 클래스 불균형 처리

메모리가 제한된 환경에 맞춰 **배치 크기(batch_size)**를 16으로 줄입니다. 나머지 로직은 이전과 동일하게 클래스를 균형 있게 샘플링하기 위해 `WeightedRandomSampler`를 사용합니다.


In [ ]:
base_dir = 'data'
train_label_dir = os.path.join(base_dir, 'train', 'train_label')
train_image_dir = os.path.join(base_dir, 'train', 'train_image')
vali_label_dir = os.path.join(base_dir, 'vali', 'vali_label')
vali_image_dir = os.path.join(base_dir, 'vali', 'vali_image')

train_samples = parse_label_files(train_label_dir, train_image_dir)
vali_samples = parse_label_files(vali_label_dir, vali_image_dir)

train_dataset = EmotionDataset(train_samples, transform=train_transform)
vali_dataset = EmotionDataset(vali_samples, transform=vali_transform)

labels = [label for _, label in train_samples]
class_sample_count = np.bincount(labels)
class_weights = 1. / class_sample_count
sample_weights = [class_weights[label] for label in labels]

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

batch_size = 16  # RTX 4070 8GB VRAM을 고려하여 배치 크기 감소
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4)
vali_loader = DataLoader(vali_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

class_counts = { '비우울(0)': int(class_sample_count[0]), '우울(1)': int(class_sample_count[1]) }
class_counts


## 6. 모델 정의와 손실 함수 설정

RTX 4070 환경에서도 높은 성능을 위해 **ResNet50** 모델을 사용하되, 혼합정밀도 학습을 적용합니다. 클래스 불균형을 해결하기 위한 가중치도 동일하게 적용합니다.


In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 혼합정밀도 학습을 위한 스케일러
scaler = GradScaler()


## 7. 혼합정밀도 학습과 검증 루프 정의

`torch.cuda.amp.autocast`와 `GradScaler`를 사용하여 학습 시 부동소수점 정밀도를 자동으로 조절합니다. 이는 GPU 메모리 사용량을 줄이고 계산 속도를 높이는 데 도움을 줍니다.


In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, scaler, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # 혼합정밀도 정방향 패스
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

        # 스케일링된 손실로 역전파
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()


## 8. 모델 학습 실행

배치 크기를 줄이고 혼합정밀도 학습을 적용하여 여러 에폭 동안 모델을 학습합니다.


In [ ]:
num_epochs = 10

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
    vali_loss, vali_acc = evaluate(model, vali_loader, criterion, device)
    print(f'Epoch {epoch}/{num_epochs}: ',
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%',
          f'| Val Loss: {vali_loss:.4f}, Val Acc: {vali_acc*100:.2f}%')


## 9. 혼동 행렬 시각화

검증 세트에 대한 혼동 행렬을 시각화하여 각 클래스의 예측 성능을 확인합니다.


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['비우울', '우울'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix (Validation Set)')
    plt.show()

# plot_confusion_matrix(model, vali_loader, device)


## 10. Grad-CAM을 이용한 시각화

모델이 이미지의 어느 부분에 주목하여 우울/비우울을 판단하는지 시각적으로 확인하기 위해 **Grad-CAM**(Gradient-weighted Class Activation Mapping)을 적용합니다. Grad-CAM은 특정 클래스에 대한 출력을 기준으로 마지막 합성곱 층의 특성맵(feature map)과 그라디언트를 결합하여 중요 영역을 강조한 히트맵을 생성합니다.

아래 코드에서는 ResNet50의 마지막 합성곱 층인 `model.layer4`에 후크(hook)를 등록하여 forward 패스에서의 feature map과 backward 패스에서의 gradient를 저장한 뒤, 이를 이용해 히트맵을 생성하고 원본 이미지 위에 오버레이합니다. 검증 데이터의 첫 번째 샘플을 예시로 사용하지만, 원하는 다른 이미지에 적용할 수 있습니다.


In [ ]:
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt

def generate_gradcam(model, input_tensor, target_class):
    model.eval()
    feature_maps = []
    gradients = []

    def forward_hook(module, input, output):
        feature_maps.append(output.detach())
    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0].detach())
    
    # 마지막 합성곱 층(layer4)에 후크 등록
    handle_f = model.layer4.register_forward_hook(forward_hook)
    handle_b = model.layer4.register_backward_hook(backward_hook)

    # forward pass
    output = model(input_tensor.unsqueeze(0))
    score = output[0, target_class]
    # backward pass
    model.zero_grad()
    score.backward()

    # 후크 해제
    handle_f.remove()
    handle_b.remove()

    # gradients와 feature_maps는 리스트로 저장되므로 첫 번째 요소 사용
    grads = gradients[0][0]  # shape: [C, H, W]
    fmap = feature_maps[0][0]  # shape: [C, H, W]

    # 각 채널별로 gradient를 평균내어 가중치 계산
    weights = grads.mean(dim=(1, 2))
    cam = torch.zeros(fmap.shape[1:], dtype=fmap.dtype).to(fmap.device)
    for i, w in enumerate(weights):
        cam += w * fmap[i]
    
    cam = torch.relu(cam)
    cam = cam - cam.min()
    cam = cam / (cam.max() + 1e-8)
    cam = cam.cpu().numpy()
    # 입력 이미지 크기로 리사이즈
    cam = np.array(Image.fromarray(cam).resize((input_tensor.size(2), input_tensor.size(1))))
    return cam


def show_gradcam_on_image(input_tensor, cam_mask, mean=IMAGENET_MEAN, std=IMAGENET_STD):
    # 입력 이미지는 정규화되어 있으므로 원래 스케일로 되돌림
    img = input_tensor.cpu().permute(1,2,0).numpy()
    img = img * np.array(std)[None, None, :] + np.array(mean)[None, None, :]
    img = np.clip(img, 0, 1)
    
    heatmap = cm.jet(cam_mask)[..., :3]  # RGBA 중 RGB만
    # 히트맵과 원본 이미지 합성
    overlay = heatmap * 0.4 + img
    overlay = overlay / overlay.max()
    
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.title('Original')
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('Grad-CAM Overlay')
    plt.imshow(overlay)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# 예시: 검증 데이터 첫 번째 샘플에 Grad-CAM 적용
test_iter = iter(vali_loader)
example_inputs, example_labels = next(test_iter)
example_input = example_inputs[0].to(device)
example_label = example_labels[0].item()

# Grad-CAM 생성
cam_mask = generate_gradcam(model, example_input, example_label)
# 시각화
show_gradcam_on_image(example_input.cpu(), cam_mask)
